## Accelerated Proximal Point Method (APPM)

In [ ]:
"""
This code tests Accelerated Proximal Point Method which is the exact optimal method that 
reduces fixed-point residual ||x - J_{alpha A}x|| respect to initial distance to solution 
for maximal monotone operator A. Introduced in "Accelerated proximal point method for 
maximally monotone operators" by Donghwan Kim (2021). This method is equivalent to what 
later came to be known as the Optimized Halpern Method, which was studied in "On the 
Convergence Rate of the Halpern Iteration" by Felix Lieder (2021). The details of the 
equivalence can be found in Exercise 12.10 of Large-Scale Convex Optimization: 
Algorithms & Analyses via Monotone Operators by Ernest K. Ryu and Wotao Yin (2022).
"""

import pepflow as pf
import numpy as np
from IPython.display import display

## Preparation: PEPFlow environment setup and context activation

In [ ]:
appm = pf.PEPContext("appm").set_as_current()
pep_builder = pf.PEPBuilder(appm)

pep_builder.clear_setup()
appm.clear()

## Input information for PEP: parameters, algorithm definition

- Problem setup,  Iteration number

In [ ]:
A = pf.declare_oper(pf.MonotoneOperator, "A")
N = 4

- Produce seqeunce generated by appm

In [ ]:
alpha = 1

x_0 = pep_builder.add_init_point("x_0")

y = x_0
for i in range(N):
    x_next = A.resolvent_step(y, alpha).add_tag(f"x_{i + 1}")
    y_next = ((i + 1) / (i + 2) * (2 * x_next - y) + 1 / (i + 2) * x_0).add_tag(
        f"y_{i + 1}"
    )
    y = y_next

- Set performance metric (and initial constraint)

In [ ]:
x_star = A.set_zero_point("x_star")
pep_builder.add_initial_constraint(
    ((x_0 - x_star) ** 2).le(1, name="initial_condition")
)

x_N = appm[f"x_{N}"]
pep_builder.set_performance_metric(A(x_N) ** 2)

In [ ]:
appm.basis_vectors()

[x_0,
 A(J_{1*A}(x_0)),
 A(J_{1*A}(y_1)),
 A(J_{1*A}(y_2)),
 A(J_{1*A}(y_3)),
 x_star]

## Solve Primal PEP

In [ ]:
result = pep_builder.solve()
print(result.opt_value)

0.06250301959936248


- Compared with expected value

In [ ]:
expected_opt_value = 1 / (alpha**2 * N**2)

print(
    "Did we guess the right closed form of convergence rate?",
    np.isclose(result.opt_value, expected_opt_value, atol=1e-3),
)

Did we guess the right closed form of convergence rate? True


- Run dashboard (this method does not require manual relaxation)

In [ ]:
pf.launch_primal_interactive(pep_builder, appm)

Dash app running on http://127.0.0.1:8050/


## Extract dual variables and numerically verify their closed forms

- Extract dual variables

In [ ]:
S_sol = result.get_gram_dual_matrix()
tau = result.dual_var_manager.dual_value("initial_condition")
lamb_sol = result.get_scalar_constraint_dual_value_in_numpy(A)

### Verify closed form expression of $\lambda$

- Print the values of $\lambda$ obtained from the solver

In [ ]:
lamb_sol.pprint()

<IPython.core.display.Math object>

- Consider proper candidate of closed form expression of $\lambda$

In [ ]:
def tag_to_index(tag, N=N):
    """This is a function that takes in a tag of an iterate and returns its index.
    We index "x_star" as "N+1 where N is the last iterate.
    """
    # Split the string on "_" and get the index
    if (idx := tag.split("_")[1]).isdigit():
        return int(idx)
    elif idx == "star":
        return N + 1

In [ ]:
def lamb(tag_i, tag_j, N=N):
    i = tag_to_index(tag_i)
    j = tag_to_index(tag_j)
    if i - 1 == j:
        if i == N + 1:
            return 2 / N  ## Between N and optimal
        else:
            return 2 * (i - 1) * i / N**2  ## Consecutive
    return 0


lamb_cand = pf.pprint_labeled_matrix(
    lamb, lamb_sol.row_names, lamb_sol.col_names, return_matrix=True
)

<IPython.core.display.Math object>

- Check whether our candidate of $\lambda$ matches with solution

In [ ]:
print(
    "Did we guess the right closed form of lambda?",
    np.allclose(lamb_cand, lamb_sol.matrix, atol=1e-3),
)

Did we guess the right closed form of lambda? True


### Closed form expression of $S$

In [ ]:
S_sol.pprint()

<IPython.core.display.Math object>

In [ ]:
appm.basis_vectors()

[x_0,
 A(J_{1*A}(x_0)),
 A(J_{1*A}(y_1)),
 A(J_{1*A}(y_2)),
 A(J_{1*A}(y_3)),
 x_star]

In [ ]:
S_guess = (A(x_N) - 1 / N * (x_0 - x_star)) ** 2

pm = pf.ExpressionManager(appm)
S_guess_eval = pm.eval_scalar(S_guess).matrix
pf.pprint_labeled_matrix(S_guess_eval, S_sol.row_names, S_sol.col_names)

<IPython.core.display.Math object>

In [ ]:
print(
    "Did we guess the right closed form of S?",
    np.allclose(S_guess_eval, S_sol.matrix, atol=1e-3),
)

Did we guess the right closed form of S? True


#### Verify symbolic calculation for fixed $N$

In [ ]:
interpolation_scalar_sum = 0
for i in range(N + 2):
    for j in range(N + 2):
        xi = "x_star" if i == N + 1 else f"x_{i}"
        xj = "x_star" if j == N + 1 else f"x_{j}"
        if lamb(xi, xj) != 0:
            interpolation_scalar_sum += lamb(xi, xj) * A.interp_ineq(xi, xj)

interpolation_scalar_sum

0+0.25*(x_2-(x_1))*(A(J_{1*A}(y_1))-A(J_{1*A}(x_0)))+0.75*(x_3-(x_2))*(A(J_{1*A}(y_2))-A(J_{1*A}(y_1)))+1.5*(x_4-(x_3))*(A(J_{1*A}(y_3))-A(J_{1*A}(y_2)))+0.5*(x_star-(x_4))*(A(x_star)-A(J_{1*A}(y_3)))

In [ ]:
RHS = -interpolation_scalar_sum - S_guess
display(RHS)

-(0+0.25*(x_2-(x_1))*(A(J_{1*A}(y_1))-A(J_{1*A}(x_0)))+0.75*(x_3-(x_2))*(A(J_{1*A}(y_2))-A(J_{1*A}(y_1)))+1.5*(x_4-(x_3))*(A(J_{1*A}(y_3))-A(J_{1*A}(y_2)))+0.5*(x_star-(x_4))*(A(x_star)-A(J_{1*A}(y_3))))-|A(J_{1*A}(y_3))-0.25*(x_0-x_star)|^2

In [ ]:
LHS = A(x_N) ** 2 - 1 / (alpha**2 * N**2) * (x_0 - x_star) ** 2
display(LHS)

|A(J_{1*A}(y_3))|^2-0.0625*|x_0-x_star|^2

In [ ]:
difference = LHS - RHS
display(difference)

|A(J_{1*A}(y_3))|^2-0.0625*|x_0-x_star|^2-(-(0+0.25*(x_2-(x_1))*(A(J_{1*A}(y_1))-A(J_{1*A}(x_0)))+0.75*(x_3-(x_2))*(A(J_{1*A}(y_2))-A(J_{1*A}(y_1)))+1.5*(x_4-(x_3))*(A(J_{1*A}(y_3))-A(J_{1*A}(y_2)))+0.5*(x_star-(x_4))*(A(x_star)-A(J_{1*A}(y_3))))-|A(J_{1*A}(y_3))-0.25*(x_0-x_star)|^2)

In [ ]:
pf.pprint_str(difference.repr_by_basis(appm))

<IPython.core.display.Math object>

\begin{align*}
    \| \tilde{\mathbb{A}}(x^N) \|^2 - \frac{\| x^0 - x^\star \|^2}{\alpha^2 N^2}
    &= -\sum_{k=1}^{N-1} \frac{k(k+1)}{\alpha^2 N^2} \langle \tilde{\mathbb{A}}(x^{k+1}) - \tilde{\mathbb{A}}(x^k), x^{k+1} - x^k \rangle \\&\quad 
    - \frac{1}{N} \langle \tilde{\mathbb{A}}(x^{N}), x^{N} - x^\star \rangle \\&\quad 
    - \| \tilde{\mathbb{A}}(x^N) - \frac{1}{\alpha N} ( x^0 - x^\star)  \| ^2.
\end{align*}